Since sqlalchemy was already installed, I don't need to pip install
This is a function to call a database using its folder path on my computer
Code taken from: https://stackoverflow.com/questions/47432988/openpyxl-read-out-excel-and-save-into-database

In [30]:
from sqlalchemy import create_engine
import pandas as pd

def initialise_database(database_path):
    db = create_engine('sqlite:///%s' % database_path)
    return db

db = initialise_database('../DatabaseDesign/PIMMS.db')               

This is a function to import data from Excel files into source tables in my PIMMS database
If the filename does not exist, it prints an error with the filename
If the source table already exists, it prints an error with the table name
Source: https://docs.python.org/3/tutorial/errors.html

In [31]:
source_names = ['AAC_Alephs', 'AAC_HMDAnalysis', 'AAC_IAMS', 'AMEMM_HMDAnalysis', 'AMEMM_IAMS', 'AMEMM_SelfAnalysis', 'DAR_ARWG', 'DAR_Assets', 'DAR_Datasets', 'FolderOverview', 'psip_DCDSpreadsheetOverview', 'psip_PreservationBids', 'SP_ProjectPlanning', 'SP_LiveWorkflow']

def import_source(sources):
    for source in sources:
        filename = '../DataSources/%s.xlsx' % source
        try:
            df = pd.read_excel(filename)
            table = 'source_%s' % source
            df.to_sql(table, db, if_exists='fail')
        except FileNotFoundError:
            print('The file called ' + filename + ' does not exist')
        except ValueError:
            print('The table called ' + table + ' already exists')

import_source(source_names)

The file called ../DataSources/AAC_Alephs.xlsx does not exist
The table called source_AAC_HMDAnalysis already exists
The table called source_AAC_IAMS already exists
The table called source_AMEMM_HMDAnalysis already exists
The table called source_AMEMM_IAMS already exists
The table called source_AMEMM_SelfAnalysis already exists
The table called source_DAR_ARWG already exists
The table called source_DAR_Assets already exists
The table called source_DAR_Datasets already exists
The table called source_FolderOverview already exists
The table called source_psip_DCDSpreadsheetOverview already exists
The table called source_psip_PreservationBids already exists
The table called source_SP_ProjectPlanning already exists
The table called source_SP_LiveWorkflow already exists


This is a function to import csv files as source tables in my PIMMS database - adjust to match the function above for excel - store excel and csv in two separate folders and add this to the paths above

In [29]:
df = pd.read_csv('../DataSources/AuditFoldersFiles/SEAsia_Files_20200207.csv')
df.to_sql('source_SEAsia_Files_20200207', db, if_exists='append')




Ignore below...

df = pd.read_excel('../DataSources/FolderOverview.xlsx')
df.to_sql('source_FolderOverview', db, if_exists='append')


These are failed attempts to turn the above code into a function to import data from data sources into source tables in my PIMMS database
I've created a list of all my data sources to loop over

def import_source(data_source_path):
    df = pd.read_excel(data_source_path)
    df.to_sql('source_amemm', db, if_exists='fail')
    db.execute('''SELECT * FROM source_ammem''').fetchall() 
    
#data_source_names = ['AMEMM_SelfAnalysis', 'AMEMM_DAR_Datasets', 'AMEMM_LegacyAnalysis', 'AsianAfrican_Aleph', 'AsianAfrican_IAMS', 'AsianAfrican_LegacyAnalysis']
#data_sources = ['../DataSources/%s.xlsx' % name for name in data_source_names]
#print(data_sources)
import_source('../DataSources/AMEMM_SelfAnalysis.xlsx')

#data_source_list = ['../DataSources/%s.xlsx' % name for name in data_source_names]
#source_tables = ['source_%s' % name for name in data_source_names]


#db.execute('''SELECT * FROM source_tables''').fetchall() 